###3.1 Vector Add

####Introducing grid-stride loop

We'll use a slight variation on the vector add code presented in a previous homework (vector_add.cu). Edit the code to build a complete vector_add program. You can refer to vector_add_solution.cu for a complete example. For this example, we have made a change to the kernel to use something called a grid-stride loop. This topic will be dealt with in more detail in a later training session, but for now we can describe it as a flexible kernel design method that allows a simple kernel to handle an arbitrary size data set with an arbitrary size "grid", i.e. the configuration of blocks and threads associated with the kernel launch. If you'd like to read more about grid-stride loops right now, you can visit https://devblogs.nvidia.com/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/


As we will see, this flexibility is important for our investigations in section 2 of this homework session. However, as before, all you need to focus on are the FIXME items, and these sections will be identical to the work you did in a previous homework assignment. If you get stuck, you can refer to the solution vector_add_solution.cu.

Note that this skeleton code includes something we didn't cover in lesson 1: CUDA error checking. Every CUDA runtime API call returns an error code. It's good practice (especially if you're having trouble) to rigorously check these error codes. A macro is given that will make this job easier. Note the special error checking method after a kernel call

##vector_add.cu

```
#include <stdio.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)


const int DSIZE = 32*1048576;
// vector add kernel: C = A + B
__global__ void vadd(const float *A, const float *B, float *C, int ds){

  for (int idx = threadIdx.x+blockDim.x*blockIdx.x; idx < ds; idx+=gridDim.x*blockDim.x)         // a grid-stride loop
    FIXME         // do the vector (element) add here
}

int main(){

  float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
  h_A = new float[DSIZE];  // allocate space for vectors in host memory
  h_B = new float[DSIZE];
  h_C = new float[DSIZE];
  for (int i = 0; i < DSIZE; i++){  // initialize vectors in host memory
    h_A[i] = rand()/(float)RAND_MAX;
    h_B[i] = rand()/(float)RAND_MAX;
    h_C[i] = 0;}
  cudaMalloc(&d_A, DSIZE*sizeof(float));  // allocate device space for vector A
  FIXME // allocate device space for vector B
  FIXME // allocate device space for vector C
  cudaCheckErrors("cudaMalloc failure"); // error checking
  // copy vector A to device:
  cudaMemcpy(d_A, h_A, DSIZE*sizeof(float), cudaMemcpyHostToDevice);
  // copy vector B to device:
  FIXME
  cudaCheckErrors("cudaMemcpy H2D failure");
  //cuda processing sequence step 1 is complete
  int blocks = 1;  // modify this line for experimentation
  int threads = 1; // modify this line for experimentation
  vadd<<<blocks, threads>>>(d_A, d_B, d_C, DSIZE);
  cudaCheckErrors("kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector C from device to host:
  FIXME
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("kernel execution failure or cudaMemcpy H2D failure");
  printf("A[0] = %f\n", h_A[0]);
  printf("B[0] = %f\n", h_B[0]);
  printf("C[0] = %f\n", h_C[0]);
  return 0;
}
```

In [10]:
%%writefile vector_add.cu
#include <stdio.h>

// error checking macro
#define cudaCheckErrors(msg) \
    do { \
        cudaError_t __err = cudaGetLastError(); \
        if (__err != cudaSuccess) { \
            fprintf(stderr, "Fatal error: %s (%s at %s:%d)\n", \
                msg, cudaGetErrorString(__err), \
                __FILE__, __LINE__); \
            fprintf(stderr, "*** FAILED - ABORTING\n"); \
            exit(1); \
        } \
    } while (0)


const int DSIZE = 32*1048576;
// vector add kernel: C = A + B
__global__ void vadd(const float *A, const float *B, float *C, int ds){

  for (int idx = threadIdx.x+blockDim.x*blockIdx.x; idx < ds; idx+=gridDim.x*blockDim.x)         // a grid-stride loop
  {// do the vector (element) add here
    C[idx] = A[idx] + B[idx];
  }
}

int main(){

  float *h_A, *h_B, *h_C, *d_A, *d_B, *d_C;
  h_A = new float[DSIZE];  // allocate space for vectors in host memory
  h_B = new float[DSIZE];
  h_C = new float[DSIZE];
  for (int i = 0; i < DSIZE; i++){  // initialize vectors in host memory
    h_A[i] = rand()/(float)RAND_MAX;
    h_B[i] = rand()/(float)RAND_MAX;
    h_C[i] = 0;}
  cudaMalloc(&d_A, DSIZE*sizeof(float));  // allocate device space for vector A
  cudaMalloc(&d_B, DSIZE*sizeof(float));  // allocate device space for vector B
  cudaMalloc(&d_C, DSIZE*sizeof(float));  // allocate device space for vector C
  cudaCheckErrors("cudaMalloc failure"); // error checking
  // copy vector A to device:
  cudaMemcpy(d_A, h_A, DSIZE*sizeof(float), cudaMemcpyHostToDevice);
  // copy vector B to device:
  cudaMemcpy(d_B, h_B, DSIZE*sizeof(float), cudaMemcpyHostToDevice);
  cudaCheckErrors("cudaMemcpy H2D failure");
  //cuda processing sequence step 1 is complete
  int blocks = 1;  // modify this line for experimentation
  int threads = 1; // modify this line for experimentation
  vadd<<<blocks, threads>>>(d_A, d_B, d_C, DSIZE);
  cudaCheckErrors("kernel launch failure");
  //cuda processing sequence step 2 is complete
  // copy vector C from device to host:
  cudaMemcpy(h_C, d_C, DSIZE*sizeof(float), cudaMemcpyDeviceToHost);
  //cuda processing sequence step 3 is complete
  cudaCheckErrors("kernel execution failure or cudaMemcpy H2D failure");
  printf("A[0] = %f\n", h_A[0]);
  printf("B[0] = %f\n", h_B[0]);
  printf("C[0] = %f\n", h_C[0]);
  return 0;
}

Overwriting vector_add.cu


In [11]:
!nvcc -arch=sm_75 vector_add.cu -o vector_add

In [12]:
!./vector_add

A[0] = 0.840188
B[0] = 0.394383
C[0] = 1.234571


Notice that the stride of the loop is blockDim.x * gridDim.x which is the total number of threads in the grid. So if there are 1280 threads in the grid, thread 0 will compute elements 0, 1280, 2560, etc. This is why I call this a grid-stride loop. By using a loop with stride equal to the grid size, we ensure that all addressing within warps is unit-stride, so we get maximum memory coalescing, just as in the monolithic version.

When launched with a grid large enough to cover all iterations of the loop, the grid-stride loop should have essentially the same instruction cost as the if statement in the monolithic kernel, because the loop increment will only be evaluated when the loop condition evaluates to true.

There are several benefits to using a grid-stride loop.

**Scalability and thread reuse.**

By using a loop, you can support any problem size even if it exceeds the largest grid size your CUDA device supports. Moreover, you can limit the number of blocks you use to tune performance. For example, it’s often useful to launch a number of blocks that is a multiple of the number of multiprocessors on the device, to balance utilization. As an example, we might launch the loop version of the kernel like this.
```
int numSMs;
cudaDeviceGetAttribute(&numSMs, cudaDevAttrMultiProcessorCount, devId);
// Perform SAXPY on 1M elements
saxpy<<<32*numSMs, 256>>>(1 << 20, 2.0, x, y);
```

When you limit the number of blocks in your grid, threads are reused for multiple computations. Thread reuse amortizes thread creation and destruction cost along with any other processing the kernel might do before or after the loop (such as thread-private or shared data initialization).

**Debugging.**

 By using a loop instead of a monolithic kernel, you can easily switch to serial processing by launching one block with one thread.

```saxpy<<<1,1>>>(1<<20, 2.0, x, y);```

This makes it easier to emulate a serial host implementation to validate results, and it can make printf debugging easier by serializing the print order. Serializing the computation also allows you to eliminate numerical variations caused by changes in the order of operations from run to run, helping you to verify that your numerics are correct before tuning the parallel version.

**Portability and readability.**

The grid-stride loop code is more like the original sequential loop code than the monolithic kernel code, making it clearer for other users. In fact we can pretty easily write a version of the kernel that compiles and runs either as a parallel CUDA kernel on the GPU or as a sequential loop on the CPU. The Hemi library provides a `grid_stride_range()` helper that makes this trivial using C++11 range-based for loops.
HEMI_LAUNCHABLE

```
void saxpy(int n, float a, float *x, float *y)
{
  for (auto i : hemi::grid_stride_range(0, n)) {
    y[i] = a * x[i] + y[i];
  }
}
```

We can launch the kernel using this code, which generates a kernel launch when compiled for CUDA, or a function call when compiled for the CPU.
```
hemi::cudaLaunch(saxpy, 1<<20, 2.0, x, y);
```
Grid-stride loops are a great way to make your CUDA kernels flexible, scalable, debuggable, and even portable. While the examples in this post have all used CUDA C/C++, the same concepts apply in other CUDA languages such as CUDA Fortran.

###3.2 Profiling Experiments

Our objective now will be to explore some of the concepts we learned in the lesson. In particular we want to see what effect grid sizing (choice of blocks, and threads per block) have on performance. We could do analysis like this using host-code-based timing methods, but we'll introduce a new concept, using a GPU profiler. In a future session, you'll learn more about the GPU profilers (Nsight Compute and Nsight Systems), but for now we will use Nsight Compute in a fairly simple fashion to get some basic data about kernel behavior, to use for comparison. (If you'd like to read more about the Nsight profilers, you can start here: https://devblogs.nvidia.com/migrating-nvidia-nsight-tools-nvvp-nvprof/)


First, note that the code has these two lines in it:
```
  int blocks = 1;  // modify this line for experimentation
  int threads = 1; // modify this line for experimentation
```
These lines control the grid sizing. The first variable blocks chooses the total number of blocks to launch. The second variable threads chooses the number of threads per block to launch. This second variable must be constrained to choices between 1 and 1024, inclusive. These are limits imposed by the GPU hardware.

Let's consider 3 cases. In each case, we will modify the blocks and threads variables, recompile the code, and then run the code under the Nsight Compute profiler.

Nsight Compute is installed as part of newer CUDA toolkits (10.1 and newer), but the path to the command line tool may or may not be set up as part of your CUDA install. Therefore it may be necessary to specify the complete command line to access the tool. We will demonstrate that here with our invocations.

For the following profiler experiments, we will assume you have loaded the profile module and acquired a node for interactive usage:





In [5]:
!nvprof ./vector_add

======== Error: application not found.
